In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn import cross_validation
import warnings
warnings.filterwarnings('ignore')

E:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import math
data_train  = pd.read_csv(r'\Users\Rex\Desktop\python\train.csv',encoding='big5')
data_test  = pd.read_csv(r'\Users\Rex\Desktop\python\test.csv',encoding='big5')
data=data_train.append(data_test)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [3]:
data['Fare'] = data['Fare'].fillna(data['Fare'].mean())
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Embarked'].fillna('S',inplace = True)
data['Cabin1'] = data['Cabin'].str.slice(0,1)
data.loc[data['Pclass']==1, 'Cabin2'] = 'A'
data.loc[data['Pclass']==2, 'Cabin2'] = 'E'
data.loc[data['Pclass']==3, 'Cabin2'] = 'F'
data['Cabin1'].fillna(data['Cabin2'],inplace = True)
#data['Cabin2'] = data.apply(lambda x: 'F' if pd.isnull(x['Cabin1']) and x['Pclass']==3 else x['Cabin1'], axis=1)
data.head()


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Cabin1,Cabin2
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,F,F
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,C,A
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,F,F
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,C,A
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,F,F


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 14 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Cabin1         1309 non-null object
Cabin2         1309 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 153.4+ KB


In [5]:
data['Ticket_info'] = data['Ticket'].apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if  x.isdigit() else 'X')
data['Ticket_info'].head()
data['Ticket2'] = data['Ticket'].apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')
data['Ticket2'].unique()

array(['A5', 'PC', 'STONO2', 'X', 'PP', 'CA', 'SCParis', 'SCA4', 'A4',
       'SP', 'SOC', 'WC', 'SOTONOQ', 'WEP', 'STONO', 'C', 'SCPARIS',
       'SOP', 'Fa', 'LINE', 'FCC', 'SWPP', 'SCOW', 'PPP', 'SC', 'SCAH',
       'AS', 'SOPP', 'FC', 'SOTONO2', 'CASOTON', 'SCA3', 'STONOQ', 'AQ4',
       'A', 'LP', 'AQ3'], dtype=object)

In [6]:
data.head(20)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Cabin1,Cabin2,Ticket_info,Ticket2
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,F,F,X,A5
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,C,A,X,PC
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,F,F,X,STONO2
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,C,A,113803,X
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,F,F,373450,X
5,28.0,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877,F,F,330877,X
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463,E,A,17463,X
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909,F,F,349909,X
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742,F,F,347742,X
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736,E,E,237736,X


In [7]:
data['Title1']=data['Name'].str.split('.',expand=True)[1]

In [8]:
data_pre=data[['Age','Cabin1','Fare','SibSp','Parch','Pclass','Sex','Survived','Embarked','PassengerId','Ticket_info','Ticket2',\
              ]]
#data_pre=data[['Age','Cabin1','Fare','family_size','Pclass','Sex','Survived','Embarked','PassengerId']]
data_pre = pd.get_dummies(data_pre,columns=['Sex','Embarked','Cabin1'])
data_pre['Ticket_info'] = data_pre['Ticket_info'].astype('category').cat.codes
data_pre['Ticket2'] = data_pre['Ticket2'].astype('category').cat.codes
#data_pre['Title1'] = data_pre['Title1'].astype('category').cat.codes
data_pre.head()

,Age,Fare,SibSp,Parch,Pclass,Survived,PassengerId,Ticket_info,Ticket2,Sex_female,...,Embarked_Q,Embarked_S,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,Cabin1_E,Cabin1_F,Cabin1_G,Cabin1_T
0,22.0,7.2500,1,0,3,0.0,1,705,2,0,...,0,1,0,0,0,0,0,1,0,0
1,38.0,71.2833,1,0,1,1.0,2,705,14,1,...,0,0,0,0,1,0,0,0,0,0
2,26.0,7.9250,0,0,3,1.0,3,705,31,1,...,0,1,0,0,0,0,0,1,0,0
3,35.0,53.1000,1,0,1,1.0,4,65,36,1,...,0,1,0,0,1,0,0,0,0,0
4,35.0,8.0500,0,0,3,0.0,5,649,36,0,...,0,1,0,0,0,0,0,1,0,0


In [9]:
data_pre = data_pre.drop('Cabin1_T',1)
data_pre = data_pre.drop('Cabin1_G',1)

datatrain = data_pre[pd.notnull(data_pre['Survived'])].sort_values(by=['PassengerId'])
datatest = data_pre[~pd.notnull(data_pre['Survived'])].sort_values(by=['PassengerId'])
datatrain1 = datatrain.drop('PassengerId',1)
#datatrain1 = datatrain1.drop('Sex_female',1)
datatest1 =  datatest.drop('PassengerId',1)
#datatest = datatest1.drop('Sex_female',1)
X_feature = datatrain1.drop('Survived',1)
y_t = datatrain['Survived']

X_feature.head()

,Age,Fare,SibSp,Parch,Pclass,Ticket_info,Ticket2,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,Cabin1_E,Cabin1_F
0,22.0,7.2500,1,0,3,705,2,0,1,0,0,1,0,0,0,0,0,1
1,38.0,71.2833,1,0,1,705,14,1,0,1,0,0,0,0,1,0,0,0
2,26.0,7.9250,0,0,3,705,31,1,0,0,0,1,0,0,0,0,0,1
3,35.0,53.1000,1,0,1,65,36,1,0,0,0,1,0,0,1,0,0,0
4,35.0,8.0500,0,0,3,649,36,0,1,0,0,1,0,0,0,0,0,1


In [10]:
X_train,X_test,y_train,y_test = train_test_split(X_feature,y_t\
,test_size=0.3,random_state = 0)
y_train.head()

857    1.0
52     1.0
386    0.0
124    0.0
578    0.0
Name: Survived, dtype: float64

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [12]:
GBDT = GradientBoostingClassifier(n_estimators=100,max_depth=3,learning_rate=0.1)
GBDT.fit(X_train,y_train)
Xgb = XGBClassifier()
Xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [13]:
from sklearn import metrics
def m_performance(X,y,clf, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True):
    y_pred=clf.predict(X)   
    if show_accuracy:
        print ("Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred)),"\n")

    if show_classification_report:
        print ("Classification report")
        print (metrics.classification_report(y,y_pred),"\n")
        
    if show_confusion_matrix:
        print ("Confusion matrix")
        print (metrics.confusion_matrix(y,y_pred),"\n")
        
m_performance(X_train,y_train,GBDT, show_classification_report=True, show_confusion_matrix=True)
m_performance(X_train,y_train,Xgb, show_classification_report=True, show_confusion_matrix=True)

Accuracy:0.913 

Classification report
             precision    recall  f1-score   support

        0.0       0.90      0.96      0.93       381
        1.0       0.94      0.83      0.88       242

avg / total       0.91      0.91      0.91       623
 

Confusion matrix
[[367  14]
 [ 40 202]] 

Accuracy:0.897 

Classification report
             precision    recall  f1-score   support

        0.0       0.90      0.94      0.92       381
        1.0       0.90      0.83      0.86       242

avg / total       0.90      0.90      0.90       623
 

Confusion matrix
[[358  23]
 [ 41 201]] 



In [14]:
from sklearn.cross_validation import cross_val_score # K折交叉验证模块

#使用K折交叉验证模块
scores = cross_val_score(GBDT, X_feature,y_t, cv=5, scoring='accuracy')
scores1 = cross_val_score(Xgb, X_feature,y_t, cv=5, scoring='accuracy')
#将5次的预测准确率打印出
print(scores.mean())
print(scores1.mean())
# 0.973333333333

#from sklearn.ensemble import RandomForestClassifier
#forest = RandomForestClassifier(criterion='entropy',n_estimators=100,random_state =2 ,n_jobs = 2)
#forest.fit(X_train,y_train)
#m_performance(X_train,y_train,forest, show_classification_report=True, show_confusion_matrix=True)

0.8351186454863434
0.8339257185028515


In [15]:
from sklearn.learning_curve import learning_curve

In [16]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt



In [17]:
import matplotlib.pyplot as plt
title = "Learning Curves (Random Forest, n_estimators = 100)"
cv = cross_validation.ShuffleSplit(datatrain.drop('Survived',1).shape[0], n_iter=10,test_size=0.3, random_state=0)
plot_learning_curve(GBDT, title, X_feature,y_t, (0.0, 1.01), cv=cv, n_jobs=4)
plot_learning_curve(Xgb, title, X_feature,y_t, (0.0, 1.01), cv=cv, n_jobs=4)
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [18]:
datatest123 = datatest1.drop('Survived',1)
datatest123.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 18 columns):
Age            418 non-null float64
Fare           418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Pclass         418 non-null int64
Ticket_info    418 non-null int16
Ticket2        418 non-null int8
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
Embarked_C     418 non-null uint8
Embarked_Q     418 non-null uint8
Embarked_S     418 non-null uint8
Cabin1_A       418 non-null uint8
Cabin1_B       418 non-null uint8
Cabin1_C       418 non-null uint8
Cabin1_D       418 non-null uint8
Cabin1_E       418 non-null uint8
Cabin1_F       418 non-null uint8
dtypes: float64(2), int16(1), int64(3), int8(1), uint8(11)
memory usage: 25.3 KB


In [19]:
submit= pd.read_csv(r'\Users\Rex\Desktop\python\gender_submission.csv',encoding='big5')
submit1= pd.read_csv(r'\Users\Rex\Desktop\python\gender_submission.csv',encoding='big5')
GBDT_res =GBDT.predict(datatest123)
submit['Survived'] = GBDT_res
submit['Survived'] = submit['Survived'].astype(int)

Xgb_res =Xgb.predict(datatest123)
submit1['Survived'] = Xgb_res
submit1['Survived'] = submit1['Survived'].astype(int)



In [20]:
%pwd
submit.to_csv(r'\Users\Rex\Desktop\python\submit_to_kaggle_titanic.csv', index= False)
submit1.to_csv(r'\Users\Rex\Desktop\python\submit_to_kaggle_titanic_xgb.csv', index= False)

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)

In [22]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [23]:
imp = sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), X_train.columns), reverse=True)
rf_imp = pd.DataFrame(imp, columns=['importances', 'features'])
rf_imp

,importances,features
0,0.1680,Age
1,0.1607,Ticket_info
2,0.1462,Fare
3,0.1334,Sex_male
4,0.1314,Sex_female
5,0.0508,Pclass
6,0.0401,SibSp
7,0.0361,Ticket2
8,0.0327,Cabin1_F
9,0.0299,Parch
